In [1]:
import pandas as pd
import numpy as np

In [26]:
cols_to_keep = ['primaryid', 'caseid', 'drug_seq',
                'role_cod', 'drugname', 'route',
                'origin', 'prod_ai', 'nda_num', 'val_vbm']

dtypes = {'primaryid': 'str',
          'caseid': 'str',
          'drug_seq': 'int8',
          'role_cod': 'str',
          'drugname': 'str',
          'route': 'str',
          'nda_num': 'str',
          'origin': 'str',
          'prod_ai': 'str',
          'val_vbm': 'int8'}

df_pd: pd.DataFrame = pd.read_csv("../data/processed/drug_table.csv",
                                      index_col=False,
                                      usecols=cols_to_keep,
                                      dtype=dtypes,
                                      low_memory=False)

In [27]:
df_pd.head()

,primaryid,caseid,drug_seq,role_cod,drugname,val_vbm,route,nda_num,origin,prod_ai
0,34483284,3448328,1,PS,SUSTIVA,1,TRANSPLACENTAL,020972,drug12q4.txt,NaN
1,34483284,3448328,2,SS,NEVIRAPINE,1,TRANSPLACENTAL,NaN,drug12q4.txt,NaN
2,34483284,3448328,3,SS,VIRACEPT,1,TRANSPLACENTAL,NaN,drug12q4.txt,NaN
3,34483284,3448328,4,SS,COMBIVIR,1,TRANSPLACENTAL,NaN,drug12q4.txt,NaN
4,34483284,3448328,5,SS,RETROVIR,1,TRANSPLACENTAL,NaN,drug12q4.txt,NaN


# VAL VBM Experiments
check if all entries with  val_vbm = 1 (which means the trade name is validated) can actually map to the correct RxNorm drugname 100%. If this is the case, we can cut down significantly on our normalization experiment data size.

In [28]:
df_val: pd.DataFrame = df_pd[df_pd["val_vbm"] == 1]  #  subset of only validated trade names

In [34]:
unique_val_drugnames = df_val["drugname"].unique()

In [37]:
unique_val_drugnames

array(['SUSTIVA', 'NEVIRAPINE', 'VIRACEPT', ...,
       'ISOSORBIDE [ISOSORBIDE DINITRATE]', 'NEOMYCIN?POLY HC',
       'TARO?TERCONAZOLE'], dtype=object)

In [29]:
import dask.dataframe as dd

In [33]:
df = dd.read_csv("../data/processed/drug_table_clean.csv", dtype=dtypes)

In [34]:
df.isna().sum().compute()

primaryid           0
caseid              0
drug_seq            0
role_cod            4
drugname         1182
route        13209398
nda_num      26309522
origin              0
prod_ai       5833978
dtype: int64

In [37]:
df[df["drugname"].isna()].compute()

,primaryid,caseid,drug_seq,role_cod,drugname,route,nda_num,origin,prod_ai
9903,73348573,7334857,2,C,NaN,NaN,NaN,drug12q4.txt,NaN
249443,87910472,8791047,6,C,NaN,NaN,NaN,drug12q4.txt,NaN
763797,93000331,9300033,9,SS,NaN,NaN,NaN,drug12q4.txt,NaN
356531,95095111,9509511,7,C,NaN,NaN,NaN,DRUG13Q1.txt,NaN
915126,96839872,9683987,3,C,NaN,NaN,NaN,DRUG13Q4.txt,NaN
...,...,...,...,...,...,...,...,...,...
453456,190769841,19076984,2,C,NaN,NaN,NaN,DRUG21Q1.txt,UNSPECIFIED INGREDIENT
456526,190778351,19077835,2,C,NaN,NaN,NaN,DRUG21Q1.txt,UNSPECIFIED INGREDIENT
456639,190778731,19077873,2,C,NaN,NaN,NaN,DRUG21Q1.txt,UNSPECIFIED INGREDIENT
456751,190779081,19077908,2,C,NaN,NaN,NaN,DRUG21Q1.txt,UNSPECIFIED INGREDIENT


In [39]:
len(df["drugname"].unique().compute())

573151

In [41]:
len(df["primaryid"].unique().compute())

10711397

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [43]:
df_pd.isna().sum()

val_vbm    0
dtype: int64

In [45]:
verbatim_idx = df_pd[df_pd["val_vbm"] == 1].index